# EVENT SAMPLER TUTORIAL

## Prerequisites 

....

### Introduction

This tutorial describes how to sampling events for an observation of a given Sky model and IRF. The main aim of the tutorial is to explain how to set the minimal configuration for the event sampling and how to obtain an output photon event list.

The core of the simulation lies into the Gammapy [`MapDatasetEventSampler`](https://docs.gammapy.org/dev/api/gammapy.cube.MapDatasetEventSampler.html) class, which is based on the inverse cumulative distribution function [(Inverse CDF)](https://en.wikipedia.org/wiki/Cumulative_distribution_function#Inverse_distribution_function_(quantile_function)). 

The `MapDatasetEventSampler` takes in input a [`Dataset`](https://docs.gammapy.org/dev/api/gammapy.datasets.Dataset.html) object containing the Sky model. The class evaluates the map of predicted counts per bin and then samples it, giving in output a set of events with true coordinates, true energies and times of arrival. It is also possible to then apply IRF corrections (i.e. PSF and energy dispersion) in order to obtain reconstructed coordinates and energies of the sampled events. 

The metadata in the event-list are then catched from an [`Observation`](https://docs.gammapy.org/dev/api/gammapy.data.Observations.html) object. 


### Setup

In [ ]:
from pathlib import Path
import numpy as np
import copy
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.data import DataStore, GTI, Observation
from gammapy.datasets import MapDataset, MapDatasetEventSampler
from gammapy.maps import MapAxis, WcsGeom, Map
from gammapy.irf import load_cta_irfs
from gammapy.makers import MapDatasetMaker
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    Model, Models, SkyModel, 
    PowerLawSpectralModel, 
    PointSpatialModel,
    GaussianSpatialModel,
    SkyDiffuseCube
)
from regions import CircleSkyRegion

### Define the Sky model: point-like source

First of all, let's define a [`Sky model`](https://docs.gammapy.org/dev/notebooks/models.html) for a point-like source centered 0.5 deg far from the Galactic Center and with a power-law spectral shape. We then save the model into a yaml file.

In [ ]:
spectral_model_pwl = PowerLawSpectralModel(index=2,amplitude="1e-12 TeV-1 cm-2 s-1", reference='1 TeV')
spatial_model_point = PointSpatialModel(lon_0= "0 deg", lat_0="0.5 deg", frame='galactic')
sky_model_pntpwl = SkyModel(spectral_model=spectral_model_pwl, spatial_model=spatial_model_point)

models_pntpwl = Models([sky_model_pntpwl])

file_model = "point-pwl.yaml"
models_pntpwl.write(file_model, overwrite=True)

### Create the dataset
In this section we show how to create the `Dataset` object needed for the event sampling (for more info about to create `Dataset` objects, please visit this [link](https://docs.gammapy.org/dev/notebooks/analysis_2.html#Preparing-reduced-datasets-geometry)). 

Hereafter, we select the IRF from the South configuration of the CTA DC1 to simulate one observation. We set the pointing position of the simulated field at the Galactic Center and we fix the exposure time to 8 hr. 

In [ ]:
IRF_FILE = "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"

POINTING = SkyCoord(0.0, 0.0, frame="galactic", unit="deg")
LIVETIME = 8 * u.hr

We then define the energy axes (true and reconstruncted), the migration axis and the geometry of the observation. 

*This is a crucial point for the correct configuration of the event sampler. Indeed the spatial and energetic binning should be treaten carefully and the finer the better. For this reason, we suggest to define the energy axes by setting a minimum binning of least 10-20 bins per decade for all the source of interest. The spatial binning may instead be different from source to source and, at first order, it should be adopted a binning significantly smaller than expected source size.*

For the examples that will be shown hereafter, we set the geometry of the dataset to a field of view of 4degx4deg and we  bin the spatial map with pixels of 0.02 deg.

In [ ]:
# dataset config
ENERGY_AXIS = MapAxis.from_energy_bounds("0.1 TeV", "100 TeV", nbin=10, per_decade=True)
ENERGY_AXIS_TRUE = MapAxis.from_energy_bounds("0.03 TeV", "300 TeV", nbin=20, per_decade=True, name="energy_true")
MIGRA_AXIS = MapAxis.from_bounds(0.5, 2, nbin=150, node_type="edges", name="migra")

WCS_GEOM = WcsGeom.create(
    skydir=POINTING, width=(4, 4), binsz=0.02, frame="galactic", axes=[ENERGY_AXIS]
)

We generate the `Dataset` object selecting the effective area, background model, the psf and the edisp from the IRF. We create an `Observation` object that containts the pointing position, the GTIs and the reference IRFs. The `Dataset` thus created can be saved into a FITS file simply just using the `write()` function.

In [ ]:
irfs = load_cta_irfs(IRF_FILE)
observation = Observation.create(
    obs_id=1001, pointing=POINTING, livetime=LIVETIME, irfs=irfs
)

empty = MapDataset.create(WCS_GEOM, energy_axis_true=ENERGY_AXIS_TRUE, migra_axis=MIGRA_AXIS)
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
dataset = maker.run(empty, observation)

dataset.write('dataset.fits.gz', overwrite=True)

### Sampling the source and background events

Now, we can finally add the `Skymodel` we want to simulate to the `Dataset` container:

In [ ]:
dataset.models.extend(models_pntpwl)
print(dataset.models)

The next step shows how to sample the events with the `MapdatasetEventSampler` class. The class requests a random number seed generator that we set with `random_state=0`, the `Dataset` and the `Observation` object. From the latter, the `MapDatasetEventSampler` class takes all the meta data information.

In [ ]:
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

The output of the event-sampler is an event list with coordinates, energies and time of arrivals of the source and background events. Source and background events are flagged by the MC_ID identifier (where 0 is the default identifier for the background).

In [ ]:
events.table

In [ ]:
print(f"Source events: {len(np.where(events.table['MC_ID']==2)[0])}")
print(f"Background events: {len(np.where(events.table['MC_ID']==0)[0])}")

We can inspect the properties of the simulated events as follows:

In [ ]:
events.peek()

Let's write the photon event list to a FITS file:

In [ ]:
events.table.write("events_0001.fits.gz", overwrite=True)

### Generate a skymap
A skymap of the simulated events is the following:

In [ ]:
counts = Map.create(frame="galactic", skydir=(0, 0.), binsz=0.02, npix=(150, 150))
counts.fill_events(events)
counts.plot(add_cbar=True)

### Fit the simulated data
We can now check the sake of the event sampling by fitting the data (a tutorial of source fitting is [here]( https://docs.gammapy.org/dev/notebooks/analysis_2.html)). We make use of the same `Skymodel` adopted for the simulation. 
Hence, we firstly read the `Dataset` and the model file, and we fill the `Dataset` with the sampled events.

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
models_sim_point = Models.read("point-pwl.yaml")

counts = Map.from_geom(WCS_GEOM)
counts.fill_events(events)
dataset.counts = counts
dataset.models.extend(models_sim_point)

Let's fit the data and look at the results:

In [ ]:
fit = Fit([dataset])
result = fit.run(optimize_opts={"print_level": 1})
print(result)

In [ ]:
result.parameters.to_table()

The results looks great!

### Extended source

Now we can focus on the case of an extended source. For example, let's consider a source with a radial gaussian morphology: 

In [ ]:
spatial_model_gauss = GaussianSpatialModel(lon_0= "0 deg", lat_0="0 deg", sigma="0.3 deg", frame='galactic')
sky_model_pntgaus = SkyModel(spectral_model=spectral_model_pwl, spatial_model=spatial_model_gauss)

models_pntgaus = Models([sky_model_pntgaus])

file_model = "gauss-pwl.yaml"
models_pntgaus.write(file_model, overwrite=True)

We can read the `Dataset` already created and fill it with the new `Skymodel`:

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
dataset.models.extend(models_pntgaus)
print(dataset.models)

As above, we can sample the events from this `Dataset` and the `Observation` object previously defined:

In [ ]:
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

Let's inspect the events:

In [ ]:
events.peek()

In [ ]:
counts = Map.create(frame="galactic", skydir=(0, 0.), binsz=0.02, npix=(150, 150))
counts.fill_events(events)
counts.smooth(0.04 * u.deg).plot(add_cbar=True)

Again, we can fit the sampled events to compare simulation and observation:

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
models_sim_pntgaus = Models.read("gauss-pwl.yaml")

counts = Map.from_geom(WCS_GEOM)
counts.fill_events(events)
dataset.counts = counts
dataset.models.extend(models_sim_pntgaus)

In [ ]:
fit = Fit([dataset])
result = fit.run(optimize_opts={"print_level": 1})
print(result)

In [ ]:
result.parameters.to_table()

The results are excellent as all the observed parameters are consistent withing 1 sigma with the simulated ones.

### Extended source using a template
The event sampler can also work with a template `Skymodel`.
Here we use the interstellar emission model map of the Fermi 3FHL, which can found in the GAMMAPY data repository.

We proceed following the same steps showed and we finally have a look at the events properties:

In [ ]:
diffuse = SkyDiffuseCube.read(
    "$GAMMAPY_DATA/fermi-3fhl-gc/gll_iem_v06_gc.fits.gz"
)
models_diffuse = Models([diffuse])

file_model = "diffuse.yaml"
models_diffuse.write(file_model, overwrite=True)

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
dataset.models.extend(models_diffuse)
print(dataset.models)

In [ ]:
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

In [ ]:
events.peek()

### Exercises
- Change the spatial model and the spectrum of the simulated Sky model;
- Include a temporal model in the simulation